In [1]:
import numpy as np
import pandas as pd
import os
import re
from datetime import datetime
from argparse import ArgumentParser
import statsmodels.api as sm
import logging
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

## utility functions
import RT_stat_utils

In [2]:
def summary_results(output_dir, datatag, cn_g_with_scrt, cn_s_with_scrt, cell_clones, argv=None):
    
    argv=None
    if argv==None:  # columns names definition
            argv = pd.Series(['reads', 'state', 'gc','clone_id','hmmcopy',
                     'model_cn_state','model_rep_state','reads'], 
                     index=['input_col','cn_col','gc_col','clone_col','cn_prior_method',
                           'frac_rt_col','rep_col','rpm_col'])

    
    cn = pd.concat([cn_s_with_scrt, cn_g_with_scrt], ignore_index=True)

    # compute the fraction of replicated bins within each cell
    # cn = compute_cell_frac(cn, frac_rt_col=argv.frac_rt_col, rep_state_col=argv.rep_col)

    # compute autocorrelation features to see which cells are truly low quality
    cell_metrics = RT_stat_utils.compute_quality_features(cn,  rpm_col=argv.rpm_col) #rep_state_col=argv.rep_col, cn_state_col=argv.cn_col,
    cell_metrics_RT_frac = RT_stat_utils.compute_cell_frac_v2(cn, frac_rt_col='cell_frac_rep', rep_state_col='model_rep_state')
    # cell_clones = cell_clones.loc[:,['cell_id','clone_id','cell_type_status']] #,'cell_type_status'
    
    # cell_metrics.to_csv(os.path.join(output_dir, datatag+'_RT_cell_metrics.csv.gz'), index=False)
    # cell_metrics_RT_frac.to_csv(os.path.join(output_dir, datatag+'_RT_cell_metrics_RT_frac.csv.gz'), index=False)
    print('Save summary results into output dir: {0}'.format(output_dir))
    
    cell_metrics = pd.merge(cell_metrics, cell_clones, on='cell_id')
    cell_metrics_RT_frac = pd.merge(cell_metrics_RT_frac, cell_clones, on='cell_id')
    cell_metrics.to_csv(os.path.join(output_dir, datatag+'_RT_cell_metrics_clones.csv.gz'), index=False)
    cell_metrics_RT_frac.to_csv(os.path.join(output_dir, datatag+'_RT_cell_metrics_RT_frac_clones.csv.gz'), index=False)
    
    p = sns.boxplot(data=cell_metrics, x="rep_bk", y="cell_type_status", hue="clone_id") #"rep_auto"
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=5)
    outname = os.path.join(output_dir, datatag + '_rep_bk.png') 
    plt.savefig(outname, dpi=150)
    plt.close()    

    p1 = sns.boxplot(data=cell_metrics_RT_frac, x="cell_frac_rep", y="cell_type_status", hue="clone_id") #"rep_auto"
    p1.axvline(0.05)
    p1.axvline(0.95)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=5)
    outname1 = os.path.join(output_dir, datatag + '_cell_frac_rep.png') 
    plt.savefig(outname1, dpi=150)
    plt.close()   



In [3]:
def load_data(sid, results_dir, argv=None):
    cell_clones_fn = os.path.join(results_dir,sid+'_clonal_RT.csv.gz')
    input_gfn = os.path.join(results_dir,sid+'_cn_g_with_scrt.csv.gz')
    input_sfn = os.path.join(results_dir,sid+'_cn_s_with_scrt.csv.gz')
    
    cn_g_with_scrt = pd.read_csv(input_gfn,compression='gzip')
    cn_s_with_scrt = pd.read_csv(input_sfn,compression='gzip')
    print('G cells: ')
    print(cn_g_with_scrt.shape)
    print('S cells: ')
    print(cn_s_with_scrt.shape)
#     cell_clones_fn = os.path.join(input_dir, 'RT_input/A130854B_filtered_cell_clones.csv') 
    cell_clones = pd.read_csv(cell_clones_fn,compression='gzip')
    print('Cell clones file: ')
    print(cell_clones.columns)
    print(cell_clones.shape)
    
    datatag=sid
    summary_results(results_dir, datatag, cn_g_with_scrt, cn_s_with_scrt, cell_clones)                 
                     

In [4]:
obs_samples = ['SA535X4XB02498','SA535X4XB05391','SA535X4XB05462','SA535X4XB05649']
sid = 'SA535X4XB02498'
results_dir = '/home/htran/storage/datasets/metastasis_results/replication_timing/RT_results/'
for sid in obs_samples:
    load_data(sid, results_dir, argv=None)

/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  """


G cells: 
(3940810, 14)
S cells: 
(1303260, 14)
Cell clones file: 
Index(['cell_id', 'clone_id', 'library_id', 'cell_type_status'], dtype='object')
(845, 4)


/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:671: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning,


Save summary results into output dir: /home/htran/storage/datasets/metastasis_results/replication_timing/RT_results/


/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  """


...

Save summary results into output dir: /home/htran/storage/datasets/metastasis_results/replication_timing/RT_results/


/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  """


G cells: 
(3141250, 14)
S cells: 
(1242500, 14)
Cell clones file: 
Index(['cell_id', 'clone_id', 'library_id', 'cell_type_status'], dtype='object')
(1002, 4)


/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:671: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning,


Save summary results into output dir: /home/htran/storage/datasets/metastasis_results/replication_timing/RT_results/


In [17]:
cell_clones_fn = os.path.join(results_dir,sid+'_clonal_RT.csv.gz')
input_gfn = os.path.join(results_dir,sid+'_cn_g_with_scrt.csv.gz')
input_sfn = os.path.join(results_dir,sid+'_cn_s_with_scrt.csv.gz')

cn_g_with_scrt = pd.read_csv(input_gfn,compression='gzip')
cn_s_with_scrt = pd.read_csv(input_sfn,compression='gzip')
print('G cells: ')
print(cn_g_with_scrt.shape)
print('S cells: ')
print(cn_s_with_scrt.shape)
#     cell_clones_fn = os.path.join(input_dir, 'RT_input/A130854B_filtered_cell_clones.csv') 
cell_clones = pd.read_csv(cell_clones_fn,compression='gzip')
print('Cell clones file: ')
print(cell_clones.columns)
print(cell_clones.shape)

argv=None
if argv==None:  # columns names definition
        argv = pd.Series(['reads', 'state', 'gc','clone_id','hmmcopy',
                 'model_cn_state','model_rep_state','reads'], 
                 index=['input_col','cn_col','gc_col','clone_col','cn_prior_method',
                       'frac_rt_col','rep_col','rpm_col'])


cn = pd.concat([cn_s_with_scrt, cn_g_with_scrt], ignore_index=True)
print(cn.columns)
print(cn.shape)



/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


G cells: 
(3940810, 14)
S cells: 
(1303260, 14)
Cell clones file: 
Index(['cell_id', 'clone_id', 'library_id', 'cell_type_status'], dtype='object')
(845, 4)
Index(['cell_id', 'chr', 'start', 'end', 'gc', 'state', 'library_id', 'reads',
       'clone_id', 'model_cn_state', 'model_rep_state', 'model_tau', 'model_u',
       'model_rho'],
      dtype='object')
(5244070, 14)


In [18]:
# cell_metrics.columns
# cell_metrics_RT_frac.columns
# cell_clones.columns
# cell_metrics_RT_frac.shape
# cn_s_with_scrt.shape
cell_metrics = RT_stat_utils.compute_quality_features(cn,  rpm_col=argv.rpm_col) #rep_state_col=argv.rep_col, cn_state_col=argv.cn_col,
cell_metrics_RT_frac = RT_stat_utils.compute_cell_frac_v2(cn, frac_rt_col='cell_frac_rep', rep_state_col='model_rep_state')



/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:671: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning,


In [30]:
frac_rt_col='cell_frac_rep'
rep_state_col='model_rep_state'
cell_metrics = []
for cell_id, cell_cn in cn.groupby('cell_id'):
    temp_rep = cell_cn[rep_state_col].values
    temp_frac = sum(temp_rep) / len(temp_rep)
#     print(cell_id)
#     print(temp_frac)
#     cn.loc[cell_cn.index, frac_rt_col] = temp_frac

#     if temp_frac > 0.95 or temp_frac < 0.05:
#         cn.loc[cell_cn.index, 'extreme_cell_frac'] = True

    # temp_df = pd.DataFrame({   
    #     'cell_id': [cell_id], 'cell_frac_rep': temp_frac
    # })
#     temp_df = pd.Series([cell_id, temp_frac], index=['cell_id','cell_frac_rep'])
    temp_df = pd.DataFrame({'cell_id': [cell_id], 'cell_frac_rep': [temp_frac]})
    cell_metrics.append(temp_df)
    
cell_metrics = pd.concat(cell_metrics, ignore_index=True)    

In [45]:
import math
def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

round_up(0.925073, decimals=4)

# cell_metrics['cell_frac_rep'].values

0.9251

In [43]:
cell_metrics['cell_frac_rep'] = [round_up(r, decimals=4) for r in cell_metrics['cell_frac_rep'].values]

In [2]:
output_dir='/home/htran/storage/datasets/metastasis_results/replication_timing/RT_results/'
cn_s_with_scrt = pd.read_csv(os.path.join(output_dir,'SA535X4XB05649_cn_s_with_scrt.csv.gz'))
print(cn_s_with_scrt.shape)
print(cn_s_with_scrt.columns)

(1242500, 14)
Index(['cell_id', 'chr', 'start', 'end', 'gc', 'state', 'library_id', 'reads',
       'clone_id', 'model_cn_state', 'model_rep_state', 'model_tau', 'model_u',
       'model_rho'],
      dtype='object')


/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
cn_s_with_scrt_extracted = cn_s_with_scrt.loc[:,['cell_id','clone_id','library_id']]

cn_s_with_scrt_extracted = cn_s_with_scrt_extracted.groupby(['cell_id','clone_id','library_id'], as_index=False).size()
cn_s_with_scrt_extracted

,cell_id,clone_id,library_id,size
0,SA535X4XB05649-A98165A-R03-C17,T,A98165A,4375
1,SA535X4XB05649-A98165A-R03-C22,I,A98165A,4375
2,SA535X4XB05649-A98165A-R03-C23,Q,A98165A,4375
3,SA535X4XB05649-A98165A-R03-C25,T,A98165A,4375
4,SA535X4XB05649-A98165A-R03-C29,R,A98165A,4375
...,...,...,...,...
279,SA535X4XB05649-A98261A-R26-C22,T,A98261A,4375
280,SA535X4XB05649-A98261A-R27-C09,N,A98261A,4375
281,SA535X4XB05649-A98261A-R28-C19,N,A98261A,4375
282,SA535X4XB05649-A98277B-R46-C69,Q,A98277B,4375


In [7]:
# cn_s_with_scrt_extracted['cell_type_status'] = np.repeat('cn_s',cn_s_with_scrt_extracted.shape[0])
cn_s_with_scrt_extracted['size'] = 

In [9]:
cn_g_with_scrt_extracted = cn_s_with_scrt_extracted

In [10]:
clonal_df = pd.concat([cn_s_with_scrt_extracted, cn_g_with_scrt_extracted], ignore_index=True)

In [1]:
# import re 
import RT_stat_utils
import run_RT

In [2]:
import numpy as np
import pandas as pd
import os
from argparse import ArgumentParser
import statsmodels.api as sm

import scdna_replication_tools
## Some issues with import funcs from scdna_replication_tools
## to fix it temporarilly, I import all funcs here
from scdna_replication_tools.cncluster import kmeans_cluster
from scdna_replication_tools.compute_consensus_clone_profiles import compute_consensus_clone_profiles
from scdna_replication_tools.assign_s_to_clones import assign_s_to_clones
from scdna_replication_tools.bulk_gc_correction import bulk_g1_gc_correction
from scdna_replication_tools.normalize_by_cell import normalize_by_cell
from scdna_replication_tools.normalize_by_clone import normalize_by_clone
from scdna_replication_tools.binarize_rt_profiles import binarize_profiles
from scdna_replication_tools.compute_pseudobulk_rt_profiles import compute_pseudobulk_rt_profiles
from scdna_replication_tools.calculate_twidth import compute_time_from_scheduled_column, calculate_twidth
from scdna_replication_tools.pert_model import pyro_infer_scRT
from scdna_replication_tools.infer_scRT import scRT #main func



In [7]:
input_dir = "/home/htran/storage/raw_DLP/metastasis_DLP/SA919/A130854B/"
input_gfn = os.path.join(input_dir, 'hmmcopy/filtered_reads_RT_g_cells.csv.gz') 
input_sfn = os.path.join(input_dir, 'hmmcopy/filtered_reads_RT_s_cells.csv.gz') 
cell_clones_fn = os.path.join(input_dir, 'RT_input/A130854B_filtered_cell_clones_v2.csv') 
nb_iterations = 2
output_dir = None

In [8]:
import re
def load_data(input_gfn, input_sfn, cell_clones_fn, argv=None):
    input_dir = os.path.dirname(input_gfn)
#     input_gfn = os.path.join(input_dir, 'hmmcopy/filtered_reads_RT_g_cells.csv.gz') 
#     input_sfn = os.path.join(input_dir, 'hmmcopy/filtered_reads_RT_s_cells.csv.gz') 
    if bool(re.search(".gz$", input_gfn)):
        cn_g = pd.read_csv(input_gfn,compression='gzip')
    else:
        cn_g = pd.read_csv(input_gfn) #header=0, index_col=0

    if bool(re.search(".gz$", input_sfn)):
        cn_s = pd.read_csv(input_sfn,compression='gzip')
    else:
        cn_s = pd.read_csv(input_sfn) #header=0, index_col=0

    
    cn_g = pd.read_csv(input_gfn,compression='gzip')
    cn_s = pd.read_csv(input_sfn,compression='gzip')
    print('G cells: ')
    print(cn_g.shape)
    print('S cells: ')
    print(cn_s.shape)
#     cell_clones_fn = os.path.join(input_dir, 'RT_input/A130854B_filtered_cell_clones.csv') 
    cell_clones = pd.read_csv(cell_clones_fn)
    print('Cell clones file: ')
    print(cell_clones.columns)
    print(cell_clones.shape)
#     select_values = cn_g['cell_id'].values
#     cells_clone_cut = cell_clones.query('cell_id in @select_values')  
# #     cells_clone_cut.index = cells_clone_cut['cell_id'].values
#     # cells_clone_cut.loc['AT13696-A130854B-R48-C07',:]
#     cells_clone_cut = cells_clone_cut.loc[:,['cell_id','clone_id', 'library_id']]
#     cell_clones = cell_clones.loc[:,['cell_id','clone_id', 'library_id','cell_type_status']]
    # included in input files
#     cn_s['library_id'] = np.repeat(libary_id,cn_s.shape[0])
#     cn_g['library_id'] = np.repeat(libary_id,cn_g.shape[0])
    cn_g = pd.merge(cn_g, cell_clones, on='cell_id')
    cn_s = pd.merge(cn_s, cell_clones, on='cell_id')
    print('G cells: ')
    print(cn_g.shape)
    print('S cells: ')
    print(cn_s.shape)
    
    if argv==None:
        argv = pd.Series(['reads', 'state', 'gc','clone_id','hmmcopy',
                 'model_cn_state','model_rep_state','reads'], 
                 index=['input_col','cn_col','gc_col','clone_col','cn_prior_method',
                       'frac_rt_col','rep_col','rpm_col'])
    # temp_cn_s = cn_s[['cell_id', 'chr', 'start', 'end', 'gc', 'state', 'library_id', 'reads']]
#     temp_cn_g = cn_g1[['cell_id', 'chr', 'start', 'end', 'gc', 'state', 'library_id', 'clone_id','reads']]
    temp_cn_s = cn_s[['cell_id', 'chr', 'start', 'end', argv.gc_col, argv.cn_col, 'library_id', argv.input_col]]
    temp_cn_g = cn_g[['cell_id', 'chr', 'start', 'end', argv.gc_col, argv.cn_col, 'library_id', argv.clone_col, argv.input_col]]
    print('G cells input: {0} {1}'.format(temp_cn_g.shape[0],temp_cn_g.shape[1]))
    print('S cells input: {0} {1}'.format(temp_cn_s.shape[0],temp_cn_s.shape[1]))
    
    return cell_clones, input_dir, temp_cn_g, temp_cn_s



cell_clones, input_dir, temp_cn_g, temp_cn_s = load_data(input_gfn, input_sfn, cell_clones_fn)

G cells: 
(3093125, 7)
S cells: 
(1295000, 7)
Cell clones file: 
Index(['umap1', 'umap2', 'cell_id', 'clone_id', 'cell_type_status',
       'library_id'],
      dtype='object')
(1003, 6)
G cells: 
(3093125, 12)
S cells: 
(1295000, 12)
G cells input: 3093125 9
S cells input: 1295000 8


In [14]:
def run_RT(datatag, input_dir, temp_cn_g, temp_cn_s, output_dir=None, nb_iterations = 500, argv=None):
    datatag = datatag.replace(' ', '_')
    if output_dir==None:
        output_dir = os.path.join(input_dir,datatag+'_RT_results')
    if not os.path.exists(output_dir): 
        os.makedirs(output_dir) 
        print('Creating an output dir: {0}'.format(output_dir))
        
#     nb_iterations = 10 #1500
    if argv==None:
        argv = pd.Series(['reads', 'state', 'gc','clone_id','hmmcopy',
                 'model_cn_state','model_rep_state','reads'], 
                 index=['input_col','cn_col','gc_col','clone_col','cn_prior_method',
                       'frac_rt_col','rep_col','rpm_col'])
    print('Creating scrt object')
    # create SPF object with input
    scrt = scRT(temp_cn_s, temp_cn_g, input_col=argv.input_col, rt_prior_col=None, assign_col=argv.cn_col,
                cn_state_col=argv.cn_col, gc_col=argv.gc_col, cn_prior_method=argv.cn_prior_method, max_iter=nb_iterations)
    # run inference
    print('Running inference')
    cn_s_with_scrt, supp_s_output, cn_g_with_scrt, supp_g_output = scrt.infer_pyro_model()
    
    # cn_s_with_scrt, supp_s_output, cn_g_with_scrt, supp_g_output
    cn_s_with_scrt.to_csv(os.path.join(output_dir,datatag+'_cn_s_with_scrt.csv.gz'))
    supp_s_output.to_csv(os.path.join(output_dir,datatag+'_supp_s_output.csv.gz'))
    cn_g_with_scrt.to_csv(os.path.join(output_dir,datatag+'_cn_g_with_scrt.csv.gz'))
    supp_g_output.to_csv(os.path.join(output_dir,datatag+'_supp_g_output.csv.gz'))
    print('Save results into output dir: {0}'.format(output_dir))
    return output_dir, cn_g_with_scrt, cn_s_with_scrt

In [15]:
datatag = 'SA919'
output_dir, cn_g_with_scrt, cn_s_with_scrt = run_RT(datatag, input_dir, temp_cn_g, temp_cn_s, output_dir, int(nb_iterations))

creating scrt object
running inference


  1461126 Loading data
  1461126 ----------------------------------------


cn_s after assigning to clones
                           cell_id chr     start       end        gc  state  \
0        AT13696-A130854B-R48-C09   1   2000001   2500000  0.594508      2   
1        AT13696-A130854B-R48-C09   1   3000001   3500000  0.584572      2   
2        AT13696-A130854B-R48-C09   1   4000001   4500000  0.482574      2   
3        AT13696-A130854B-R48-C09   1   4500001   5000000  0.481828      2   
4        AT13696-A130854B-R48-C09   1   5000001   5500000  0.447446      2   
...                           ...  ..       ...       ...       ...    ...   
1294995  AT13696-A130854B-R68-C70   Y  18000001  18500000  0.380458      0   
1294996  AT13696-A130854B-R68-C70   Y  18500001  19000000  0.369494      0   
1294997  AT13696-A130854B-R68-C70   Y  20500001  21000000  0.386036      0   
1294998  AT13696-A130854B-R68-C70   Y  21000001  21500000  0.395094      0   
1294999  AT13696-A130854B-R68-C70   Y  21500001  22000000  0.381986      0   

        library_id  reads clone

  1627828 Start inference for G1-phase cells.
/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/pyro/poutine/subsample_messenger.py:171: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if len(shape) >= -dim and shape[dim] != 1:
/home/htran/Projects/hakwoo_project/testing_methods/scdna_replication_tools/src/scdna-replication-tools/scdna_replication_tools/pert_model.py:595: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  lamb = pyro.param('expose_lambda', torch.tensor([lambda_init]), constraint=constraint

Save results into output dir: /home/htran/storage/raw_DLP/metastasis_DLP/SA919/A130854B/hmmcopy/SA919_RT_results


In [17]:
# '/home/htran/storage/raw_DLP/metastasis_DLP/SA919/A130854B/hmmcopy/SA919_RT_results/'
import matplotlib.pyplot as plt
argv=None
if argv==None:
        argv = pd.Series(['reads', 'state', 'gc','clone_id','hmmcopy',
                 'model_cn_state','model_rep_state','reads'], 
                 index=['input_col','cn_col','gc_col','clone_col','cn_prior_method',
                       'frac_rt_col','rep_col','rpm_col'])
cn = pd.concat([cn_s_with_scrt, cn_g_with_scrt], ignore_index=True)

# compute the fraction of replicated bins within each cell
# cn = compute_cell_frac(cn, frac_rt_col=argv.frac_rt_col, rep_state_col=argv.rep_col)

# compute autocorrelation features to see which cells are truly low quality
cell_metrics = RT_stat_utils.compute_quality_features(cn,  rpm_col=argv.rpm_col) #rep_state_col=argv.rep_col, cn_state_col=argv.cn_col,
cell_metrics_RT_frac = RT_stat_utils.compute_cell_frac_v2(cn, frac_rt_col='cell_frac_rep', rep_state_col='model_rep_state')
cell_clones = cell_clones.loc[:,['cell_id','clone_id','cell_type_status']]
cell_metrics = pd.merge(cell_metrics, cell_clones, on='cell_id')

cell_metrics_RT_frac = pd.merge(cell_metrics_RT_frac, cell_clones, on='cell_id')



/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:671: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning,
/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:677: RuntimeWarning: invalid value encountered in true_divide
  acf = avf[: nlags + 1] / avf[0]
  3975027 findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.


findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.


  3975028 findfont: score(<Font 'DejaVu Sans Display' (DejaVuSansDisplay.ttf) normal normal 400 normal>) = 10.05


findfont: score(<Font 'DejaVu Sans Display' (DejaVuSansDisplay.ttf) normal normal 400 normal>) = 10.05


  3975030 findfont: score(<Font 'cmsy10' (cmsy10.ttf) normal normal 400 normal>) = 10.05


...

findfont: score(<Font 'DejaVu Sans Mono' (DejaVuSansMono-Bold.ttf) normal normal 700 normal>) = 10.335


  3975099 findfont: score(<Font 'DejaVu Sans Mono' (DejaVuSansMono.ttf) normal normal 400 normal>) = 10.05


findfont: score(<Font 'DejaVu Sans Mono' (DejaVuSansMono.ttf) normal normal 400 normal>) = 10.05


  3975100 findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0 to DejaVu Sans ('/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf') with score of 0.050000.


findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0 to DejaVu Sans ('/home/htran/anaconda3/envs/myPython37/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf') with score of 0.050000.


In [20]:
import seaborn as sns
p = sns.boxplot(data=cell_metrics, x="rep_bk", y="cell_type_status", hue="clone_id") #"rep_auto"
outname = os.path.join(output_dir, datatag + '_rep_bk.png') 
plt.savefig(outname, dpi=150)
plt.close()    

p1 = sns.boxplot(data=cell_metrics_RT_frac, x="cell_frac_rep", y="cell_type_status", hue="clone_id") #"rep_auto"
p1.axvline(0.05)
p1.axvline(0.95)
outname1 = os.path.join(output_dir, datatag + '_cell_frac_rep.png') 
plt.savefig(outname1, dpi=150)
plt.close()    

In [19]:
output_dir

'/home/htran/storage/raw_DLP/metastasis_DLP/SA919/A130854B/hmmcopy/SA919_RT_results'

In [21]:
input_dir = "/home/htran/storage/raw_DLP/metastasis_DLP/SA919/A130854B/RT_input/"
input_gfn = os.path.join(input_dir, 'filtered_reads_RT_g_cells.csv.gz') 
input_sfn = os.path.join(input_dir, 'filtered_reads_RT_s_cells.csv.gz') 
cell_clones_fn = os.path.join(input_dir, 'A130854B_filtered_cell_clones_v2.csv') 
nb_iterations = 2
output_dir = None
datatag = 'SA919'
print(input_gfn)
print(input_sfn)
print(cell_clones_fn)

/home/htran/storage/raw_DLP/metastasis_DLP/SA919/A130854B/RT_input/filtered_reads_RT_g_cells.csv.gz
/home/htran/storage/raw_DLP/metastasis_DLP/SA919/A130854B/RT_input/filtered_reads_RT_s_cells.csv.gz
/home/htran/storage/raw_DLP/metastasis_DLP/SA919/A130854B/RT_input/A130854B_filtered_cell_clones_v2.csv


In [22]:
from datetime import datetime
start = datetime.now()

In [24]:
end = datetime.now()

td = (end - start).total_seconds()/60 

print(f"The time of execution of above program is : {td:.03f} mins")

The time of execution of above program is : 0.708ms


In [25]:
import logging
logging.basicConfig(filename=os.path.join(input_dir, 'log.txt'), encoding='utf-8', level=logging.DEBUG)
logging.debug('This message should go to the log file')
logging.info('So should this')
logging.info(f"The time of execution of above program is : {td:.03f} mins")
logging.warning('And this, too')
logging.error('And non-ASCII stuff, too, like Øresund and Malmö')


  6133569 This message should go to the log file


This message should go to the log file


  6133571 So should this
  6133572 The time of execution of above program is : 0.708 mins
  6133573 And this, too
  6133574 And non-ASCII stuff, too, like Øresund and Malmö
